# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Análisis de Grandes Volúmenes de Datos**

### Tecnológico de Monterrey

### Profesor: Dr. Iván Olmos Pineda

## Actividad Semana 6

### **Aprendizaje supervisado y no supervisado**

### A01796679

# 1. **Introducción teórica**:


El aprendizaje automático (Machine Learning) es una rama de la inteligencia artificial que permite a las computadoras aprender a partir de datos y tomar decisiones sin estar explícitamente programadas para cada tarea. En términos generales, los algoritmos de aprendizaje automático se clasifican en dos grandes grupos: aprendizaje supervisado y aprendizaje no supervisado.

 ## **Aprendizaje Supervisado**

El aprendizaje supervisado se basa en el uso de un conjunto de datos etiquetado, donde cada observación incluye una serie de características (variables independientes) y una etiqueta o valor objetivo (variable dependiente). El objetivo es que el algoritmo aprenda una función que relacione las características con las etiquetas, para luego poder realizar predicciones sobre datos nuevos.

Entre los algoritmos más representativos en la literatura se encuentran:

- Regresión Lineal y Regresión Logística
- Árboles de Decisión (Decision Trees)
- Bosques Aleatorios (Random Forest)
- Gradient Boosted Trees (GBTClassifier)
- Redes Neuronales Multicapa (Multilayer Perceptron)
- Máquinas de Vectores de Soporte (SVM)

En el contexto de PySpark, una herramienta de procesamiento distribuido sobre Apache Spark, los algoritmos supervisados disponibles a través del módulo pyspark.ml (MLlib) incluyen:

- DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
- MultilayerPerceptronClassifier
- LogisticRegression, LinearRegression
- NaiveBayes


## **Aprendizaje No Supervisado**

En el aprendizaje no supervisado no se utilizan etiquetas; el objetivo es descubrir estructuras ocultas, patrones o agrupamientos en los datos. Es comúnmente utilizado en tareas como segmentación de clientes, reducción de dimensionalidad y análisis exploratorio.

Algunos algoritmos representativos son:

- K-Means
- Gaussian Mixture Models
- Clustering Jerárquico
- Análisis de Componentes Principales (PCA)
- Modelos de Tópicos como LDA

En PySpark, se pueden aplicar varios algoritmos no supervisados a través de pyspark.ml.clustering y pyspark.ml.feature, entre ellos:

- KMeans
- GaussianMixture
- PowerIterationClustering (PIC)
- PCA (para reducción de dimensionalidad)
- LDA (Latent Dirichlet Allocation, para modelado de temas)


# 2. **Selección de los datos**:



Se seleccionó el dataset "Chicago Crimes - 2001 to Present", el cual contiene información
detallada sobre crímenes reportados en la ciudad de Chicago desde el año 2001 hasta la
actualidad. Ahora bien, el volumen y la riqueza de atributos de este dataset permiten
trabajar con un escenario real de Big Data, donde el procesamiento eficiente de grandes
cantidades de información es fundamental. El **objetivo** de esta actividad es **aplicar algoritmos de aprendizaje supervisado y no supervisado mediante PySpark** para la resolución de problemas en análisis de datos, fomentando el desarrollo de habilidades prácticas en el manejo y procesamiento eficiente de grandes conjuntos de datos.

In [101]:
#librerias
from pyspark import SparkContext
from pyspark.sql import SparkSession

from pyspark.sql.functions import col, when, isnan, count, udf,sum, isnull
from pyspark.sql.types import IntegerType, DoubleType
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import to_timestamp, year, month, dayofweek

from pyspark.ml.feature import StringIndexer, VectorAssembler

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator




In [58]:
# Detenemos cualquier sesión de Spark activa para evitar conflictos
if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()

# Iniciamos una nueva SparkSession
spark = SparkSession.builder \
    .appName("Police Crimes") \
    .master("local[*]") \
    .getOrCreate()

El dataset contiene más de 8 millones de registros de incidentes criminales ocurridos en la
ciudad de Chicago, recopilados por el Departamento de Policía. Cada registro describe un
evento único, proporcionando múltiples atributos que facilitan un análisis profundo de la
actividad delictiva.

In [59]:
# Cargamos el archivo CSV como DataFrame de Spark
ruta = "/content/Chicago_Crimes_-_2001_to_Present.csv"
df = spark.read.csv(ruta, header=True, inferSchema=True)
df.show(5)

+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|      ID|Case Number|                Date|               Block|IUCR|Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|
+--------+-----------+--------------------+--------------------+----+------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+
|10224738|   HY411648|09/05/2015 01:30:...|     043XX S WOOD ST|0486|     BATTERY|DOMESTIC BATTERY ...|           RESIDENCE| false|    true| 924|   

In [60]:
# Inspeccionamos el esquema inferido automáticamente
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [61]:
# Resumen estadístico de todas las variables
df.describe().show()

+-------+------------------+------------------+--------------------+--------------+------------------+-----------------+---------------+--------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+--------------------+-------------------+--------------------+--------------------+
|summary|                ID|       Case Number|                Date|         Block|              IUCR|     Primary Type|    Description|Location Description|              Beat|          District|              Ward|    Community Area|          FBI Code|     X Coordinate|      Y Coordinate|              Year|          Updated On|           Latitude|           Longitude|            Location|
+-------+------------------+------------------+--------------------+--------------+------------------+-----------------+---------------+--------------------+------------------+------------------+------------------+--

In [62]:
# Imprimiendo los tipos de datos del Dataframe
df.dtypes

[('ID', 'int'),
 ('Case Number', 'string'),
 ('Date', 'string'),
 ('Block', 'string'),
 ('IUCR', 'string'),
 ('Primary Type', 'string'),
 ('Description', 'string'),
 ('Location Description', 'string'),
 ('Arrest', 'boolean'),
 ('Domestic', 'boolean'),
 ('Beat', 'int'),
 ('District', 'int'),
 ('Ward', 'int'),
 ('Community Area', 'int'),
 ('FBI Code', 'string'),
 ('X Coordinate', 'int'),
 ('Y Coordinate', 'int'),
 ('Year', 'int'),
 ('Updated On', 'string'),
 ('Latitude', 'double'),
 ('Longitude', 'double'),
 ('Location', 'string')]

In [63]:
print("Número de registros: " + str(df.count()))
print("Número de columnas: " + str(len(df.columns)))

Número de registros: 7811711
Número de columnas: 22


En la siguiente celda podemos observar que la mayoría de las columnas cuentan con datos
completos o casi completos, lo cual es positivo para el análisis. Las columnas ´Ward´ y
´Community Area’ presentan la mayor cantidad de valores nulos, con cerca de 7.8% de los
registros afectados. También, las coordenadas geográficas(X, Y, Latitud, Longitud y
Location) muestran un porcentaje ligeramente superior al 1% de datos faltantes. Será
importante prestar atención a estos valores nulos durante la limpieza y el análisis futuro
para evitar que impacten negativamente en los resultados, especialmente en estudios
relacionados con segmentación geográfica o análisis por áreas administrativas.

In [72]:
# Generamos una expresión para cada columna para contar nulos
null_counts = df.select([
    sum(col(c).isNull().cast("int")).alias(c) for c in df.columns
])

null_counts.show()

+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+------+--------------+--------+------------+------------+----+----------+--------+---------+--------+
| ID|Case Number|Date|Block|IUCR|Primary Type|Description|Location Description|Arrest|Domestic|Beat|District|  Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On|Latitude|Longitude|Location|
+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+------+--------------+--------+------------+------------+----+----------+--------+---------+--------+
|  0|          4|   0|    0|   0|           0|          0|               10558|     0|       0|   0|      47|614848|        613476|       0|       87465|       87465|   0|         0|   87465|    87465|   87465|
+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+------+--------------+--------+------------+---

Para garantizar que cada partición tenga una muestra representativa de los datos
disponibles, se utilizará la técnica de muestreo aleatorio estratificado con fracción fija
(fraction) en PySpark. Esta técnica asegura que dentro de cada subconjunto de datos se
mantenga la proporción de ocurrencias, evitando sesgos por sobre o subrepresentación.

In [65]:
spark = SparkSession.builder \
    .appName("Chicago Crime Partitioning") \
    .master("local[*]") \
    .getOrCreate()

ruta = "/content/Chicago_Crimes_-_2001_to_Present.csv"

df = spark.read.csv(ruta, header=True, inferSchema=True)

# Aplicamos reglas de particionamiento
regla_A = df.filter((col("Primary Type") == "THEFT") & (col("Domestic") == True))
regla_B = df.filter((col("Primary Type") == "BATTERY") & (col("Domestic") == False))
regla_C = df.filter((col("Primary Type") == "NARCOTICS") & (col("Domestic") == True))
regla_D = df.filter((col("Primary Type") == "ASSAULT") & (col("Domestic") == False))

# Mostramos el tamaño de cada conjunto completo
print("Tamaño total por regla:")
print("Regla A:", regla_A.count())
print("Regla B:", regla_B.count())
print("Regla C:", regla_C.count())
print("Regla D:", regla_D.count())


Tamaño total por regla:
Regla A: 44739
Regla B: 801552
Regla C: 312
Regla D: 392983


Para garantizar que cada partición tenga una muestra representativa de los datos
disponibles, se utilizará la técnica de muestreo aleatorio estratificado con fracción fija
(fraction) en PySpark. Esta técnica asegura que dentro de cada subconjunto de datos se
mantenga la proporción de ocurrencias, evitando sesgos por sobre o subrepresentación.

In [66]:
# Extraemos una muestra del 10% con semilla fija
muestra_A = regla_A.sample(fraction=0.1, seed=42)
muestra_B = regla_B.sample(fraction=0.1, seed=42)
muestra_C = regla_C.sample(fraction=0.1, seed=42)
muestra_D = regla_D.sample(fraction=0.1, seed=42)

# Mostramos el tamaño de cada muestra
print("\nTamaño de la muestra por regla:")
print("Muestra A:", muestra_A.count())
print("Muestra B:", muestra_B.count())
print("Muestra C:", muestra_C.count())
print("Muestra D:", muestra_D.count())


Tamaño de la muestra por regla:
Muestra A: 4391
Muestra B: 80409
Muestra C: 40
Muestra D: 39349


# 3. **Preparación de los datos**:
En esta etapa, se deberán de aplicar estrategias de corrección sobre los datos que integran a la muestra M que se ha preparado en el paso previo, de tal forma que de deje un conjunto M listo para ser procesado por los algoritmos de aprendizaje a aplicar. Para ello se deben de considerar pasos como: corrección de registros / columnas con valores nulos, identificación de valores atípicos, transformación de los tipos de datos, etc. Con lo anterior, se tendrá una muestra M pre-procesada.

In [80]:
# Unimos todas las muestras en un solo DataFrame
muestra_unificada = muestra_A.union(muestra_B).union(muestra_C).union(muestra_D)

# Verificamos valores nulos
nulos = muestra_unificada.select([
    sum(col(c).isNull().cast("int")).alias(c) for c in muestra_unificada.columns
])
nulos.show()

+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+-----+--------------+--------+------------+------------+----+----------+--------+---------+--------+
| ID|Case Number|Date|Block|IUCR|Primary Type|Description|Location Description|Arrest|Domestic|Beat|District| Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On|Latitude|Longitude|Location|
+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+-----+--------------+--------+------------+------------+----+----------+--------+---------+--------+
|  0|          0|   0|    0|   0|           0|          0|                   0|     0|       0|   0|       1|10750|         10762|       0|         592|         592|   0|         0|     592|      592|     592|
+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+-----+--------------+--------+------------+--------

In [81]:
# Variables de entrada y objetivo
columnas_modelo = ['Primary Type', 'Domestic', 'District', 'Community Area', 'Arrest']  # 'Arrest' como variable objetivo

# Filtramos columnas
df_modelo = muestra_unificada.select(columnas_modelo)

# Eliminamos filas con valores nulos en estas columnas
df_modelo = df_modelo.dropna()

In [85]:
# Codificamos 'Primary Type'
indexador_tipo = StringIndexer(inputCol='Primary Type', outputCol='PrimaryTypeIndex')
df_modelo = indexador_tipo.fit(df_modelo).transform(df_modelo)

# Codificamos 'Arrest' (target)
df_modelo = df_modelo.withColumn("Arrest_str", col("Arrest").cast("string"))
indexador_arresto = StringIndexer(inputCol='Arrest_str', outputCol='label')
df_modelo = indexador_arresto.fit(df_modelo).transform(df_modelo)

In [88]:
#Ensamblamos las características en una sola columna 'features'
ensamblador = VectorAssembler(
    inputCols=['PrimaryTypeIndex', 'Domestic', 'District', 'Community Area'],
    outputCol='features'
)
df_final = ensamblador.transform(df_modelo).select('features', 'label')

# 4. **Preparación del conjunto de entrenamiento y prueba**:
Para dividir la muestra `M` en conjuntos de entrenamiento y prueba, utilicé la función `randomSplit()` de PySpark con una proporción del 70% para entrenamiento y 30% para prueba, estableciendo una semilla fija (`seed=42`) para garantizar la reproducibilidad de los resultados.

Elegí esta técnica de muestreo aleatorio simple porque, al haber construido previamente la muestra `M` mediante un muestreo estratificado sobre reglas bien definidas (combinando tipo de crimen y naturaleza doméstica del delito), puedo asegurar que ya se mantiene la representatividad dentro del conjunto total. El muestreo aleatorio posterior me permite evitar sesgos adicionales durante la asignación a los subconjuntos de entrenamiento y prueba.

Decidí usar la proporción 70/30 porque es una práctica común en aprendizaje automático, ya que proporciona un buen equilibrio: por un lado, me asegura un conjunto de entrenamiento suficientemente grande como para que el modelo generalice adecuadamente, y por otro, me permite contar con un conjunto de prueba significativo para evaluar el desempeño del modelo de forma confiable.


In [89]:
train_data, test_data = df_final.randomSplit([0.7, 0.3], seed=42)
print("Tamaño entrenamiento:", train_data.count())
print("Tamaño prueba:", test_data.count())

Tamaño entrenamiento: 79375
Tamaño prueba: 34051


# 5. **Construcción de modelos de aprendizaje supervisado y no supervisado**:
Para este punto realizarás dos experimentos separados, dónde se aplicará un algoritmo de aprendizaje supervisado y uno de aprendizaje no supervisado sobre la muestra M. Para el caso de aprendizaje supervisado, se deberá de identificar cuál es la variable objetivo (columna) de aprendizaje, mientras que, para el caso de aprendizaje no supervisado, se debe de seleccionar todas las columnas que se desean considerar como características bajo las cuales se realizará el proceso de agrupamiento. Usando las implementaciones correspondientes de PySpark, se deberá de ejecutar el aprendizaje correspondiente a partir de la invocación de las funciones respectivas. Para este ejercicio, se deberá seleccionar un criterio básico para medir la calidad del resultado obtenido, dependiendo de cada tipo de aprendizaje implementado. La elección quedará a juicio de cada estudiante.


- Algoritmo de aprendizaje Supervisado:

In [92]:

# Definimos el modelo
rf = RandomForestClassifier(featuresCol='features', labelCol='label', numTrees=100, seed=42)

# Entrenamos el modelo con el conjunto de entrenamiento
modelo_rf = rf.fit(train_data)

# Aplicamos el modelo al conjunto de prueba
predicciones = modelo_rf.transform(test_data)

# Mostramos algunas predicciones
predicciones.select("prediction", "label", "features").show(5)


+----------+-----+------------------+
|prediction|label|          features|
+----------+-----+------------------+
|       0.0|  0.0|[2.0,1.0,1.0,32.0]|
|       0.0|  0.0|[2.0,1.0,1.0,35.0]|
|       0.0|  0.0|[2.0,1.0,2.0,35.0]|
|       0.0|  1.0|[2.0,1.0,2.0,35.0]|
|       0.0|  0.0|[2.0,1.0,2.0,36.0]|
+----------+-----+------------------+
only showing top 5 rows



In [93]:
# Evaluador usando precisión
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
precision = evaluator.evaluate(predicciones)

# F1-score
f1_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1_score = f1_evaluator.evaluate(predicciones)

print(f"Precisión del modelo: {precision:.4f}")
print(f"F1-score del modelo: {f1_score:.4f}")

Precisión del modelo: 0.7881
F1-score del modelo: 0.6950


In [94]:
# Evaluación en el conjunto de entrenamiento
train_predictions = modelo_rf.transform(train_data)

train_accuracy = evaluator.evaluate(train_predictions)
train_f1 = f1_evaluator.evaluate(train_predictions)

print(f"Precisión en entrenamiento: {train_accuracy:.4f}")
print(f"F1-score en entrenamiento: {train_f1:.4f}")


Precisión en entrenamiento: 0.7860
F1-score en entrenamiento: 0.6922


Conclusion: Mi modelo no presenta síntomas de sobreentrenamiento ni subentrenamiento. Los valores de precisión y F1 son muy similares entre los conjuntos de entrenamiento y prueba, con diferencias menores al 1%, lo que indica que el modelo generaliza correctamente sin sobreajustarse ni fallar en captar patrones relevantes.

- Algoritmo de aprendizaje no Supervisado

In [95]:
# Selección de variables para clustering
columnas_clustering = ['PrimaryTypeIndex', 'Domestic', 'District', 'Community Area']

# Ensamblamos características
ensamblador_kmeans = VectorAssembler(inputCols=columnas_clustering, outputCol='features')
df_kmeans = ensamblador_kmeans.transform(df_modelo).select('features')

In [98]:
# Definimos y entrenamos el modelo
kmeans = KMeans(k=4, seed=42)
modelo_kmeans = kmeans.fit(df_kmeans)

# Aplicamos el modelo
predicciones_kmeans = modelo_kmeans.transform(df_kmeans)

In [ ]:
# Evaluación usando el índice de silueta (Silhouette Score)
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='features', metricName='silhouette')
silhouette = evaluator.evaluate(predicciones_kmeans)


In [103]:
print(f"Índice de silueta: {silhouette:.4f}")

Índice de silueta: 0.7554


Conclusión:

Entrené un modelo de agrupamiento K-Means utilizando las variables codificadas: tipo de crimen, si fue doméstico, distrito y área comunitaria. Seleccioné inicialmente 4 clústeres para explorar la estructura oculta en los datos.

El modelo logró un **índice de silueta de 0.7554**, lo cual indica que los clústeres generados están bien definidos y presentan una buena separación entre sí. Este valor sugiere que el modelo fue capaz de identificar patrones latentes significativos en los datos, agrupando registros con características similares de manera efectiva.

Este resultado me parece bastante positivo, considerando la naturaleza compleja y multidimensional del fenómeno criminal. Para refinar aún más el análisis, podría experimentar con distintos valores de `k` y observar cómo varía el índice de silueta para elegir el número óptimo de clústeres.


# 6. FINAL

**Comparación entre los modelos supervisado y no supervisado**

Después de aplicar ambos enfoques de aprendizaje, considero que el modelo **supervisado (Random Forest)** y el **no supervisado (K-Means)** ofrecieron perspectivas complementarias sobre los datos.

El modelo **supervisado** alcanzó una **precisión del 78.81%** y un **F1-score de 69.50%**, lo cual indica un buen desempeño al predecir si se realizó un arresto en función de variables como tipo de crimen, distrito, domesticidad y área comunitaria. Además, la similitud entre los resultados de entrenamiento y prueba sugiere que el modelo generaliza bien y no presenta sobreajuste.

Por otro lado, el modelo **no supervisado (K-Means)** logró un **índice de silueta de 0.7554**, lo que indica que los clústeres están bien definidos. Este resultado es bastante sólido en términos de agrupamiento, lo que demuestra que el modelo fue capaz de identificar patrones latentes y agrupaciones significativas en los datos sin necesidad de etiquetas.

En resumen, considero que **ambos modelos fueron exitosos en sus respectivos objetivos**, pero si tuviera que elegir uno por su **impacto práctico**, me inclinaría por el **modelo supervisado**. Esto se debe a que permite una interpretación directa y útil para decisiones reales, como la predicción de arrestos. Sin embargo, el análisis no supervisado también aportó valor al revelar segmentaciones útiles dentro del conjunto de datos.
